In [207]:
import pandas as pd
import numpy as np
import os, sys
import csv
pd.set_option('display.max_colwidth', -1)

In [ ]:
orig_file = "/mnt/tweets/dev-parsed.csv"
cln_file = "/mnt/tweets/dev-parsed-cleaned.csv"
dict_file = ""
labeled_file = "../data/parsed/dev-labeled.csv"

Clean the data. Due to the bug of Pandas `to_csv` (https://github.com/pandas-dev/pandas/issues/17365), we have the following lines that `read_csv` cannot correctly parse:

```
24,135724218678657024,"Thanking the man above for letting me & my loved ones see another day &lt;3
#thankful"
25,148946558874947584,No one wants yogurt on a Monday night. :( #lonely
```

In [395]:
with open(orig_file) as fd:
    lines = fd.readlines()
    lines_cleaned = []

    i = 0
    
    def iter_func(i, lines):
        '''
        recursively read the next line, stop when the next line starts with a number
        '''
        
        cur_l = lines[i]
        if not cur_l[0].isdigit():
            lx, pos = iter_func(i+1, lines)
            return cur_l.strip()+" "+lx, pos
        else:
            return "", i
    
    while i < len(lines)-1:
        if lines[i][0].isdigit() and not lines[i+1][0].isdigit():
            cur_l = lines[i]
            lx, i = iter_func(i+1, lines)
            lines_cleaned.append(cur_l.strip()+" "+lx)
            continue
        elif lines[i][0].isdigit() and lines[i+1][0].isdigit():
            lines_cleaned.append(lines[i].strip())
            i += 1
            continue
        else:
            '''
            Unexpected lines
            '''
            print(i, lines[i])
            break
    
with open("/mnt/tweets/dev-parsed-cleaned.csv", 'w') as fd:
    for line in lines_cleaned:
        fd.write(line+"\n")

Build a dictionary to obtain the labels of tweets

In [380]:
data = pd.read_csv(cln_file, names=['id', 'tid', 'tweet'])

In [383]:
raw_data = pd.read_csv(dict_file, delimiter='\t', names=['t_id', 'emotion'])
r_data_dict = raw_data.set_index('t_id').to_dict()

In [387]:
data['emotion'] = data['tid'].apply(lambda tid: r_data_dict['emotion'].get(tid, np.NaN))
data = data.drop('id', axis=1)
data = data.reset_index(drop=True)

In [268]:
data.to_csv(labeled_file)